<a href="https://colab.research.google.com/github/Meepyss/superstore_etl_power_bi/blob/main/Data_Cleaning_SampleSuperstore.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install kaggle

In [ ]:
from google.colab import files
files.upload()  # selecione o kaggle.json baixado


In [58]:
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json


In [ ]:
!pip install kaggle pandas numpy matplotlib


In [60]:
!kaggle datasets download -d vivek468/superstore-dataset-final --unzip


Dataset URL: https://www.kaggle.com/datasets/vivek468/superstore-dataset-final
License(s): other


In [61]:
!ls -lh


total 3.0M
-rw-r--r-- 1 root root  55K Apr 22 00:20  dim_cliente.csv
-rw-r--r-- 1 root root  15K Apr 22 00:20  dim_data.csv
-rw-r--r-- 1 root root  89K Apr 22 00:20  dim_produto.csv
-rw-r--r-- 1 root root   77 Apr 22 00:20  dim_shipmode.csv
-rw-r--r-- 1 root root 138K Apr 22 00:20  fact_vendas.csv
-rw-r--r-- 1 root root   63 Apr 22 00:22 'kaggle (1).json'
-rw-r--r-- 1 root root   63 Apr 22 00:06  kaggle.json
drwxr-xr-x 1 root root 4.0K Apr 17 13:36  sample_data
-rw-r--r-- 1 root root 2.2M Apr 22 00:22 'Sample - Superstore.csv'
-rw-r--r-- 1 root root 417K Apr 22 00:20  superstore_cleaned.csv


In [62]:
import pandas as pd

def load_data(filepath="Sample - Superstore.csv", encoding="ISO-8859-1"):
    return pd.read_csv(filepath, encoding=encoding)

In [63]:
print(df.shape)      # ex.: (9994, 21)
print(df.info())     # tipos e nulos
print(df.head())     # primeiras linhas


(1686, 27)
<class 'pandas.core.frame.DataFrame'>
Index: 1686 entries, 3 to 9983
Data columns (total 27 columns):
 #   Column         Non-Null Count  Dtype         
---  ------         --------------  -----         
 0   Row ID         1686 non-null   int64         
 1   Order ID       1686 non-null   object        
 2   Order Date     1686 non-null   datetime64[ns]
 3   Ship Date      1686 non-null   datetime64[ns]
 4   ShipMode       1686 non-null   object        
 5   Customer ID    1686 non-null   object        
 6   Customer Name  1686 non-null   object        
 7   Segment        1686 non-null   object        
 8   Country        1686 non-null   object        
 9   City           1686 non-null   object        
 10  State          1686 non-null   object        
 11  Postal Code    1686 non-null   int64         
 12  Region         1686 non-null   object        
 13  Product ID     1686 non-null   object        
 14  Category       1686 non-null   object        
 15  Sub-Category   

In [64]:
def clean_data(df):
    """
    Remove pedidos cancelados, converte datas, adiciona colunas de tempo,
    drop duplicates e cria métricas de negócio.
    """
    # Remover pedidos cancelados (Order ID começando com 'C')
    df = df[~df['Order ID'].str.startswith('C')]

    # Converter colunas de data
    df['Order Date'] = pd.to_datetime(df['Order Date'], dayfirst=False)
    df['Ship Date']  = pd.to_datetime(df['Ship Date'],  dayfirst=False)

    # Extrair atributos temporais
    df = df.assign(
        Year        = df['Order Date'].dt.year,
        Month       = df['Order Date'].dt.month,
        Quarter     = df['Order Date'].dt.quarter,
        DayOfWeek   = df['Order Date'].dt.dayofweek
    )

    # Remover duplicatas
    df = df.drop_duplicates(subset=['Row ID'], keep='first')

    # Criar métricas de negócio
    df['AvgOrderValue'] = df['Sales'] / df['Quantity']
    df['DeliveryDays']  = (df['Ship Date'] - df['Order Date']).dt.days

    # Ajustar nome de coluna para dimensão
    df = df.rename(columns={'Ship Mode':'ShipMode'})
    return df


Para evitar que retornos ou cancelamentos distorçam métricas de vendas, excluímos todas as linhas cujo Order ID comece com “C” (indicativo de cancelamento):

O Series.dt expõe propriedades temporais como .year, .month, .quarter e .dayofweek
Pandas
.
O método DataFrame.assign() permite acrescentar múltiplas colunas de uma só vez

In [65]:
df.isnull().sum()


,0
Row ID,0
Order ID,0
Order Date,0
Ship Date,0
ShipMode,0
Customer ID,0
Customer Name,0
Segment,0
Country,0
City,0


In [66]:
def clean_data(df):
    """
    Remove pedidos cancelados, converte datas, adiciona colunas de tempo,
    drop duplicates e cria métricas de negócio.
    """
    # Remover pedidos cancelados (Order ID começando com 'C')
    df = df[~df['Order ID'].str.startswith('C')]

    # Converter colunas de data
    df['Order Date'] = pd.to_datetime(df['Order Date'], dayfirst=False)
    df['Ship Date']  = pd.to_datetime(df['Ship Date'],  dayfirst=False)

    # Extrair atributos temporais
    df = df.assign(
        Year        = df['Order Date'].dt.year,
        Month       = df['Order Date'].dt.month,
        Quarter     = df['Order Date'].dt.quarter,
        DayOfWeek   = df['Order Date'].dt.dayofweek
    )

    # Remover duplicatas
    df = df.drop_duplicates(subset=['Row ID'], keep='first')

    # Criar métricas de negócio
    df['AvgOrderValue'] = df['Sales'] / df['Quantity']
    df['DeliveryDays']  = (df['Ship Date'] - df['Order Date']).dt.days

    # Ajustar nome de coluna para dimensão
    df = df.rename(columns={'Ship Mode':'ShipMode'})
    return df


In [67]:
print(df.shape)
print(df.dtypes)
print(df[['AvgOrderValue','DeliveryDays']].describe())


(1686, 27)
Row ID                    int64
Order ID                 object
Order Date       datetime64[ns]
Ship Date        datetime64[ns]
ShipMode                 object
Customer ID              object
Customer Name            object
Segment                  object
Country                  object
City                     object
State                    object
Postal Code               int64
Region                   object
Product ID               object
Category                 object
Sub-Category             object
Product Name             object
Sales                   float64
Quantity                  int64
Discount                float64
Profit                  float64
Year                      int32
Month                     int32
Quarter                   int32
DayOfWeek                 int32
AvgOrderValue           float64
DeliveryDays              int64
dtype: object
       AvgOrderValue  DeliveryDays
count    1686.000000   1686.000000
mean       57.820501      3.953737
std   

In [68]:
def create_dimensions(df):
    """
    Cria as dimensões DimData, DimProduto, DimCliente e DimShipMode,
    atribuindo surrogate keys.
    """
    # DimData
    dim_data = (
        df[['Order Date','Year','Month','Quarter','DayOfWeek']]
        .drop_duplicates()
        .reset_index(drop=True)
    )
    dim_data['DateID'] = dim_data.index + 1

    # DimProduto
    dim_produto = (
        df[['Product ID','Category','Sub-Category','Product Name']]
        .drop_duplicates()
        .rename(columns={'Product ID':'ProductID','Product Name':'ProductName'})
        .reset_index(drop=True)
    )
    dim_produto['ProductID_Key'] = dim_produto.index + 1

    # DimCliente
    dim_cliente = (
        df[['Customer ID','Segment','Country','State','City','Postal Code','Region']]
        .drop_duplicates()
        .rename(columns={'Customer ID':'CustomerID','Postal Code':'PostalCode'})
        .reset_index(drop=True)
    )
    dim_cliente['CustomerID_Key'] = dim_cliente.index + 1

    # DimShipMode
    dim_ship = (
        df[['ShipMode']]
        .drop_duplicates()
        .reset_index(drop=True)
    )
    dim_ship['ShipModeID'] = dim_ship.index + 1

    return dim_data, dim_produto, dim_cliente, dim_ship

In [48]:
# 4. Criar tabela de fatos

def create_fact(df, dim_data, dim_produto, dim_cliente, dim_ship):
    """
    Gera a tabela FactVendas unindo chaves surrogate das dimensões
    e selecionando as medidas.
    """
    fact_vendas = (
        df
        .merge(dim_data,    on=['Order Date','Year','Month','Quarter','DayOfWeek'])
        .merge(dim_produto, left_on='Product ID', right_on='ProductID')
        .merge(dim_cliente, left_on='Customer ID', right_on='CustomerID')
        .merge(dim_ship,    on='ShipMode')
        [[
            'DateID',
            'ProductID_Key',
            'CustomerID_Key',
            'ShipModeID',
            'Sales',
            'Profit',
            'Quantity',
            'AvgOrderValue',
            'DeliveryDays'
        ]]
        .rename(columns={
            'ProductID_Key':'ProductID',
            'CustomerID_Key':'CustomerID'
        })
    )
    return fact_vendas


In [69]:
def export_data(df_clean, dim_data, dim_produto, dim_cliente, dim_ship, fact_vendas):
    """
    Salva todos os DataFrames em arquivos CSV para consumo pelo Power BI.
    """
    df_clean.to_csv("superstore_cleaned.csv", index=False)
    dim_data.to_csv("dim_data.csv", index=False)
    dim_produto.to_csv("dim_produto.csv", index=False)
    dim_cliente.to_csv("dim_cliente.csv", index=False)
    dim_ship.to_csv("dim_shipmode.csv", index=False)
    fact_vendas.to_csv("fact_vendas.csv", index=False)

In [70]:
# 6. Fluxo principal

def main():
    # 6.1 Carrega e limpa
    df = load_data()
    df_clean = clean_data(df)

    # 6.2 Cria dimensões e fato
    dim_data, dim_produto, dim_cliente, dim_ship = create_dimensions(df_clean)
    fact_vendas = create_fact(df_clean, dim_data, dim_produto, dim_cliente, dim_ship)

    # 6.3 Exporta CSVs
    export_data(df_clean, dim_data, dim_produto, dim_cliente, dim_ship, fact_vendas)

    print("ETL completo! Arquivos CSV gerados:")
    print(" - superstore_cleaned.csv")
    print(" - dim_data.csv")
    print(" - dim_produto.csv")
    print(" - dim_cliente.csv")
    print(" - dim_shipmode.csv")
    print(" - fact_vendas.csv")

if __name__ == "__main__":
    main()

ETL completo! Arquivos CSV gerados:
 - superstore_cleaned.csv
 - dim_data.csv
 - dim_produto.csv
 - dim_cliente.csv
 - dim_shipmode.csv
 - fact_vendas.csv


<ipython-input-66-6b5b44666f08>:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Order Date'] = pd.to_datetime(df['Order Date'], dayfirst=False)
<ipython-input-66-6b5b44666f08>:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Ship Date']  = pd.to_datetime(df['Ship Date'],  dayfirst=False)
